**Step 1 - Cleaning the Financial Data**
Load 'compustat financials.pickle Download compustat financials.pickle'.  This data provides fields from companies' financial statements.  The data dictionary is in the file "ratios for corporate defaults.ipynb".

Refer to the ratios notebook and it's data dictionary.  You can find the overall data dictionary in the data module.  Within one week, recommend to me at least one extra field from this dictionary for download and explain why you think it could be related to credit risk.  I will add these fields and upload the new data on Thursday.
Create the new fields as you see in the ratios notebook as well as new ratios based on the new fields downloaded.
This step requires judgment on your part. If we don't normalize most of the factors, they will all behave as size factors because all fields get bigger as companies get bigger.  Choose either total assets or revenues/sales as a size variable.  For the rest, divide by an appropriate denominator (for example, total assets).  

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
from dateutil.relativedelta import relativedelta

financials = pd.read_pickle('/content/drive/MyDrive/compustat financials.pickle')
financials['total_assets'] = financials['at']
del financials['at']
financials = financials[~pd.isnull(financials.total_assets)]

ratings = pd.read_pickle('/content/drive/MyDrive/S&P ratings (3).pickle')
names = ratings[['gvkey', 'entity_pname']].copy()
names = names.drop_duplicates()
ratings = ratings[['gvkey', 'ratingdate', 'ratingsymbol']]
ratings = ratings[~ratings.ratingsymbol.isin(['NR', 'SD', 'R', 'D', 'C'])]
ratings = ratings.drop_duplicates()
ratings = ratings.sort_values(['gvkey', 'ratingdate'], ascending = [True, False])

info = pd.read_pickle('/content/drive/MyDrive/company info (1).pickle')
info = info[['gvkey', 'gsector']]
info = info[~info.gsector.isna()]

financials = financials.merge(info, on = 'gvkey')
ratings = ratings.merge(info, on = 'gvkey')

gvkeys = list(set(financials.gvkey).intersection(set(ratings.gvkey)))
financials = financials[financials.gvkey.isin(gvkeys)]
ratings = ratings[ratings.gvkey.isin(gvkeys)]

print(f'There are {len(financials):,} rows in financials')
print(f'There are {len(ratings):,} rows in raings')
print(f'There are {len(gvkeys):,} unique companies')

There are 91,621 rows in financials
There are 25,408 rows in raings
There are 5,450 unique companies


Create the new fields as you see in the ratios notebook as well as new ratios based on the new fields downloaded.

In [ ]:
#Leverage Ratios
financials['total_debt'] = financials['dlc'] + financials['dltt']
financials = financials[financials.total_debt != np.inf]
financials = financials[financials.total_debt != -np.inf]
financials = financials[(financials.total_debt >= 0) & (financials.total_debt <=100)]

financials['debt_to_equity'] = financials['total_debt'] / financials['ceq']
financials = financials[financials.debt_to_equity != np.inf]
financials = financials[financials.debt_to_equity != -np.inf]
financials = financials[(financials.debt_to_equity >= 0) & (financials.debt_to_equity <= 100)]

financials['debt_to_assets'] = financials['total_debt'] / financials['total_assets']
financials = financials[financials.debt_to_assets != np.inf]
financials = financials[financials.debt_to_assets != -np.inf]
financials = financials[(financials.debt_to_assets >= 0) & (financials.debt_to_assets <= 10)]

financials['debt_to_capital'] = financials['total_debt'] / (financials['ceq'] + financials['total_debt'])
financials = financials[financials.debt_to_capital != np.inf]
financials = financials[financials.debt_to_capital != -np.inf]
financials = financials[(financials.debt_to_capital >= 0) & (financials.debt_to_capital <= 10)]

# Coverage Ratio

financials['interest_coverage'] = financials['ebit'] / financials['xint']
financials = financials[(financials.interest_coverage != np.inf) & (financials.interest_coverage != -np.inf) ]
financials['cash_coverage'] = (financials['ebit'] + financials['dp']) / financials['xint']
financials = financials[(financials.cash_coverage != np.inf) & (financials.cash_coverage != -np.inf) ]

#Profitability Ratios

financials['gross_profit'] = (financials['sale'] - financials['cogs']) / financials['sale']

financials['operating_profit'] = financials['oibdp'] / financials['sale']

financials['net_profit'] = financials['ni'] / financials['sale']

financials['roa'] = financials['ni'] / financials['total_assets']

financials['roe'] = financials['ni'] / financials['ceq']


financials = financials[~financials.debt_to_equity.isna()]
financials = financials[~financials.debt_to_assets.isna()]
financials = financials[~financials.debt_to_capital.isna()]
financials = financials[~financials.interest_coverage.isna()]
financials = financials[~financials.cash_coverage.isna()]
financials = financials[~financials.gross_profit.isna()]
financials = financials[~financials.net_profit.isna()]
financials = financials[~financials.roe.isna()]

print(f'There are {len(financials):,} rows in financials')
financials.columns

<ipython-input-5-fbdfffb1a8af>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  financials['total_debt'] = financials['dlc'] + financials['dltt']


There are 12,338 rows in financials


Index(['gvkey', 'datadate', 'fyear', 'fyr', 'lt', 'ceq', 'act', 'lct', 'invt',
       'rect', 'ap', 'dlc', 'dltt', 'dltis', 'dvt', 'che', 'xint', 'xrd',
       'xsga', 'oibdp', 'ebit', 'ebitda', 'sale', 'cogs', 'ni', 'oancf',
       'fincf', 'csho', 'prcc_f', 'freq', 'pstk', 'dp', 'capx', 'intan',
       'gdwl', 'txt', 'dv', 'urect', 'total_assets', 'gsector', 'total_debt',
       'debt_to_equity', 'debt_to_assets', 'debt_to_capital',
       'interest_coverage', 'cash_coverage', 'gross_profit',
       'operating_profit', 'net_profit', 'roa', 'roe'],
      dtype='object')

Step 2 - Adding in the Default Data
Load the ‘S&P Ratings.pickle Download S&P Ratings.pickle’. We will combine the two datasets – pd.merge and dt.timedelta will be useful. I will provide Jupyter notebooks this week showing how to use both.  

Load the S&P Ratings data and create a table of defaults as you did for the first assignment

Merge this data with the financial data but this time use a six month offset from the statement date.  For example, if the statement date was 2003-12-31, use a window of 2004-06-30 to 2005-06-30 to capture defaults.

Each company should have at most one default so delete all records after the default date minus six months.  (so from 2004-01-01 onward)

In [ ]:
defaults = pd.read_pickle('/content/drive/MyDrive/S&P ratings (3).pickle')

defaults = defaults[defaults.ratingsymbol.isin(['D', 'SD', 'R'])]
a = defaults.groupby('gvkey', as_index = False).ratingdate.min()
defaults = defaults.merge(a, on = ['gvkey', 'ratingdate'])
defaults = defaults[['gvkey', 'ratingdate']]
defaults.columns = ['gvkey', 'default_date']

print(f'There are {len(defaults):,} defaults')
defaults.head()

There are 1,211 defaults


,gvkey,default_date
0,065345,2010-06-28
1,126136,2006-09-15
2,060839,2003-06-13
3,021701,2004-02-24
4,063755,2000-05-02


this section of the code focuses on identifying and extracting the first default date for each company from a larger dataset of credit ratings. It filters for specific default ratings, groups the data by company, finds the minimum default date, and then creates a new DataFrame containing only the company identifier and its corresponding first default date.

In [ ]:
fd = pd.merge(financials, defaults, on = 'gvkey', how = 'left')
fd.loc[fd.default_date.isna(), 'default_date'] = dt.date(2999, 12, 31)
fd = fd.sort_values(['gvkey', 'datadate'])
fd['default_minus_six_months'] = [x - relativedelta(months = 6) for x in fd.default_date]
fd = fd[fd.datadate < fd.default_minus_six_months]
del fd['default_minus_six_months']

fd['date_plus_six_months'] = [x + relativedelta(months = 7) for x in fd.datadate]
fd['date_plus_18_months'] = [x + relativedelta(months = 18) for x in fd.datadate]

fd['default_flag'] = 0
fd.loc[(fd.default_date > fd.date_plus_six_months) & (fd.default_date <= fd.date_plus_18_months), 'default_flag'] = 1
fd = fd.drop_duplicates()

print(f'There are {len(fd):,} rows in fd')
fd.default_flag.sum()
fd.columns

There are 11,969 rows in fd


Index(['gvkey', 'datadate', 'fyear', 'fyr', 'lt', 'ceq', 'act', 'lct', 'invt',
       'rect', 'ap', 'dlc', 'dltt', 'dltis', 'dvt', 'che', 'xint', 'xrd',
       'xsga', 'oibdp', 'ebit', 'ebitda', 'sale', 'cogs', 'ni', 'oancf',
       'fincf', 'csho', 'prcc_f', 'freq', 'pstk', 'dp', 'capx', 'intan',
       'gdwl', 'txt', 'dv', 'urect', 'total_assets', 'gsector', 'total_debt',
       'debt_to_equity', 'debt_to_assets', 'debt_to_capital',
       'interest_coverage', 'cash_coverage', 'gross_profit',
       'operating_profit', 'net_profit', 'roa', 'roe', 'default_date',
       'date_plus_six_months', 'date_plus_18_months', 'default_flag'],
      dtype='object')

Step 3 - Single Factor Analysis

You will normally start a project like this with one-to-two hundred potential factors. Running all of them would be untenable. We look at two different methods to explore which factors should be used in the modeling step. There are no right answers here.

Use the dataframe corr() function to look at the variable correlations. For example, if your dataframe name is data, the following will get you all correlations with the data flag.
a = data.corr()
a = a[‘default’]

Use the auc metric from scikit learn.
From sklearn import metrics.
auc = metrics.roc_auc_score(data.default, data.x) where x is the field you are interested in.

2. Choose a cutoff range where you will exclude variables from further analysis.  Keeping all variables is acceptable but you might find that the runtime is too long.  

In [ ]:
from sklearn import metrics

# Assuming 'fd' is your DataFrame and 'default_flag' is the target variable
features = ['fyr', 'lt', 'ceq', 'act', 'lct', 'invt', 'ap', 'dlc', 'dltt', 'dltis', 'dvt', 'che', 'xint', 'xrd',
       'xsga', 'oibdp', 'ebit', 'ebitda', 'sale', 'cogs', 'ni', 'oancf',
       'fincf', 'csho', 'prcc_f', 'pstk', 'dp', 'capx', 'intan',
       'gdwl', 'txt', 'dv', 'urect', 'total_assets', 'gsector',
       'total_debt', 'debt_to_equity', 'debt_to_capital',
       'debt_to_assets', 'interest_coverage', 'roa', 'roe',
       'cash_coverage', 'gross_profit', 'operating_profit', 'net_profit']

auc_scores = {}  # Store AUC scores for each feature
selected_features = []  # Store features above the cutoff

# Set the AUC cutoff range
lower_auc_cutoff = 0.2  # Adjust as needed
upper_auc_cutoff = 0.75  # Adjust as needed


for feature in features:
    # Calculate AUC for the current feature, handling NaN and infinite values
    data_for_auc = fd[['default_flag', feature]].replace([np.inf, -np.inf], np.nan).dropna()

    if len(data_for_auc) > 0:
        auc = metrics.roc_auc_score(data_for_auc['default_flag'], data_for_auc[feature])
        auc_scores[feature] = auc

        # Check if AUC is within the desired range
        if auc <= lower_auc_cutoff or auc >= upper_auc_cutoff:
            selected_features.append(feature)
    else:
        print(f"Skipping {feature} due to insufficient data after removing NaNs and infinites.")


# Print the AUC scores and selected features
for feature, auc in auc_scores.items():
    print(f"AUC for {feature}: {auc}")

# Change 'auc_cutoff' to either 'lower_auc_cutoff' or 'upper_auc_cutoff'
# depending on which one you intended to use for filtering.
print("\nSelected features (AUC <= {} or AUC >= {}):".format(lower_auc_cutoff, upper_auc_cutoff))
print(selected_features)

AUC for fyr: 0.5121985742594642
AUC for lt: 0.607328764919731
AUC for ceq: 0.4403164166044728
AUC for act: 0.4304576138384112
AUC for lct: 0.5112666247304097
AUC for invt: 0.5043389168970565
AUC for ap: 0.5667994042994043
AUC for dlc: 0.5595645572226369
AUC for dltt: 0.8108159404997812
AUC for dltis: 0.5676018178640098
AUC for dvt: 0.3651297204273144
AUC for che: 0.4468455425795451
AUC for xint: 0.7921513498211391
AUC for xrd: 0.2415243795106495
AUC for xsga: 0.4255207826340893
AUC for oibdp: 0.3010718789407314
AUC for ebit: 0.28970970481509123
AUC for ebitda: 0.3010718789407314
AUC for sale: 0.4710090845922228
AUC for cogs: 0.5178860951694675
AUC for ni: 0.2852767841058239
AUC for oancf: 0.3271096880647442
AUC for fincf: 0.6607948259722577
AUC for csho: 0.6281279758755687
AUC for prcc_f: 0.22635049854852957
AUC for pstk: 0.5175129320328382
AUC for dp: 0.49161026327302676
AUC for capx: 0.49279009126466755
AUC for intan: 0.5099615789799838
AUC for gdwl: 0.5194972826086957
AUC for txt: 0

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


Step 4 - Multi-factor Analysis

Use itertools.combinations to generate all possible four-factor combinations of the factors that you generated in Step 3.
Cycle through every combination and generate a logistic regression using Statsmodels for each combination.

Make sure the ones column is used for every regression so you have a constant / intercept.

Don’t keep any regression where the correlation between any two factors is higher than 50%.

Examine the top three regressions. Are they significantly different from each other by pseudo-R-squared?

For the top three regressions, calculate the predicted PDs for all observations. Use this to calculate the AUC for each. Are they significantly different? If they are not significantly different, are they qualitatively different? Put your comments about the models in the code.

Choose one of the top three regressions and print its summary.

In [ ]:
import itertools
import statsmodels.api as sm
import numpy as np  # Import numpy if not already imported

# Assume 'selected_features' from Step 3 contains the filtered features
# Example: selected_features = ['debt_to_equity', 'debt_to_capital', 'debt_to_assets', 'interest_coverage', 'roa']

# Generate all 4-factor combinations
for combination in itertools.combinations(selected_features, 4):
    # Create independent variables (X) for the current combination
    X = fd[list(combination)]

    # Replace infinite values with NaN
    X = X.replace([np.inf, -np.inf], np.nan)

    # Drop rows with any missing values in X or the target variable
    X = X.dropna()

    # Filter 'fd' to match rows in X
    filtered_fd = fd.loc[X.index]

    X = sm.add_constant(X)  # Add constant term

    # Create and fit the logistic regression model
    model = sm.Logit(filtered_fd['default_flag'], X).fit()

    # Print model summary
    print(f"Combination: {combination}")
    print(model.summary())
    print("-" * 50)  # Separator between models

Optimization terminated successfully.
         Current function value: 0.007572
         Iterations 12
Combination: ('dltt', 'xint', 'total_debt', 'debt_to_equity')
                           Logit Regression Results                           
Dep. Variable:           default_flag   No. Observations:                11969
Model:                          Logit   Df Residuals:                    11964
Method:                           MLE   Df Model:                            4
Date:                Mon, 24 Mar 2025   Pseudo R-squ.:                  0.1091
Time:                        14:07:55   Log-Likelihood:                -90.625
converged:                       True   LL-Null:                       -101.72
Covariance Type:            nonrobust   LLR p-value:                 0.0001837
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const             -9.0610      0.848 

In [ ]:
# Store regression results
regression_results = []

# Generate all 15-factor combinations (modified from 4 to 15 based on your code)
for combination in itertools.combinations(selected_features, 4):
    # Create independent variables (X) for the current combination
    X = fd[list(combination)]

    # Replace infinite values with NaN
    X = X.replace([np.inf, -np.inf], np.nan)

    # Drop rows with any missing values in X or the target variable
    X = X.dropna()

    # Filter 'fd' to match rows in X
    filtered_fd = fd.loc[X.index]

    X = sm.add_constant(X)  # Add constant term

    # Create and fit the logistic regression model
    try:
        model = sm.Logit(filtered_fd['default_flag'], X).fit(disp=0)  # disp=0 to suppress convergence messages

        # Store results
        regression_results.append({
            'combination': combination,
            'model': model,
            'pseudo_r_squared': model.prsquared
        })
    except Exception as e:
        print(f"Error fitting model for combination {combination}: {e}")


# Sort regression results by pseudo-R-squared in descending order
regression_results.sort(key=lambda x: x['pseudo_r_squared'], reverse=True)

# Get the top three regressions
top_three_regressions = regression_results[:3]

# Examine the top three regressions
print("\nTop Three Regressions:")
for i, result in enumerate(top_three_regressions):
    print(f"\nRegression {i + 1}:")
    print(f"Combination: {result['combination']}")
    print(f"Pseudo-R-squared: {result['pseudo_r_squared']}")

    # Calculate predicted PDs and AUC
    X_all = fd[list(result['combination'])]  # Use all observations for prediction
    X_all = X_all.replace([np.inf, -np.inf], np.nan)  # Handle infinite values
    X_all = X_all.fillna(X_all.mean())  # Impute missing values (replace with mean)
    X_all = sm.add_constant(X_all)
    predicted_probs = result['model'].predict(X_all)
    auc = metrics.roc_auc_score(fd['default_flag'], predicted_probs)
    print(f"AUC: {auc}")

    result['predicted_probs'] = predicted_probs  # Store predicted probabilities
    result['auc'] = auc  # Store AUC


# Compare pseudo-R-squared values
print("\nComparison of Pseudo-R-squared:")
for i in range(len(top_three_regressions)):
    for j in range(i + 1, len(top_three_regressions)):
        diff = abs(top_three_regressions[i]['pseudo_r_squared'] - top_three_regressions[j]['pseudo_r_squared'])
        print(f"Difference between Regression {i + 1} and Regression {j + 1}: {diff}")
        # Add your interpretation of the significance of the difference here

# Compare AUC values
print("\nComparison of AUC:")
for i in range(len(top_three_regressions)):
    for j in range(i + 1, len(top_three_regressions)):
        diff = abs(top_three_regressions[i]['auc'] - top_three_regressions[j]['auc'])
        print(f"Difference between Regression {i + 1} and Regression {j + 1}: {diff}")
        # Add your interpretation of the significance of the difference here

# Qualitative analysis of models (add your comments here)


Top Three Regressions:

Regression 1:
Combination: ('dltt', 'xint', 'total_debt', 'debt_to_capital')
Pseudo-R-squared: 0.1161814387307406
AUC: 0.8468294001080886

Regression 2:
Combination: ('dltt', 'total_debt', 'debt_to_equity', 'debt_to_capital')
Pseudo-R-squared: 0.11366750988489771
AUC: 0.8441143165967522

Regression 3:
Combination: ('dltt', 'xint', 'debt_to_equity', 'debt_to_capital')
Pseudo-R-squared: 0.11247984072120598
AUC: 0.8425830609671359

Comparison of Pseudo-R-squared:
Difference between Regression 1 and Regression 2: 0.0025139288458428943
Difference between Regression 1 and Regression 3: 0.0037015980095346235
Difference between Regression 2 and Regression 3: 0.0011876691636917291

Comparison of AUC:
Difference between Regression 1 and Regression 2: 0.0027150835113364247
Difference between Regression 1 and Regression 3: 0.004246339140952737
Difference between Regression 2 and Regression 3: 0.0015312556296163127


In [ ]:
# ... (previous code remains the same) ...

# Choose one of the top three regressions and print its summary
# Choosing Regression 1 (with the highest pseudo-R-squared)
print("\nChosen Regression Summary (Regression 1):")
print(top_three_regressions[0]['model'].summary())


Chosen Regression Summary (Regression 1):
                           Logit Regression Results                           
Dep. Variable:           default_flag   No. Observations:                11969
Model:                          Logit   Df Residuals:                    11964
Method:                           MLE   Df Model:                            4
Date:                Mon, 24 Mar 2025   Pseudo R-squ.:                  0.1162
Time:                        14:10:47   Log-Likelihood:                -89.902
converged:                       True   LL-Null:                       -101.72
Covariance Type:            nonrobust   LLR p-value:                 9.448e-05
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -9.3943      0.920    -10.206      0.000     -11.198      -7.590
dltt                0.0167      0.020      0.827      0.408      -0.023  

For your chosen model, provide the following information:

1. Average predicted PD for defaulted records and non-defaulted records.

2. For the variables in your chosen model, provide average values for defaulted and non-defaulted records.

In [ ]:

# Regression 1 analysis
regression_1_result = top_three_regressions[0]
predicted_probs = regression_1_result['predicted_probs']

# Average predicted PD for defaulted and non-defaulted records
avg_pd_defaulted = predicted_probs[fd['default_flag'] == 1].mean()
avg_pd_non_defaulted = predicted_probs[fd['default_flag'] == 0].mean()

print("\nRegression 1 - Average Predicted PD:")
print(f"Defaulted Records: {avg_pd_defaulted:.4f}")
print(f"Non-Defaulted Records: {avg_pd_non_defaulted:.4f}")

# Average values of variables for defaulted and non-defaulted records
variables = list(regression_1_result['combination'])
print("\nRegression 1 - Average Values of Variables:")
for var in variables:
    avg_defaulted = fd.loc[fd['default_flag'] == 1, var].mean()
    avg_non_defaulted = fd.loc[fd['default_flag'] == 0, var].mean()
    print(f"\nVariable: {var}")
    print(f"Defaulted Records: {avg_defaulted:.4f}")
    print(f"Non-Defaulted Records: {avg_non_defaulted:.4f}")

# ... (rest of the code remains the same) ...


Regression 1 - Average Predicted PD:
Defaulted Records: 0.0045
Non-Defaulted Records: 0.0011

Regression 1 - Average Values of Variables:

Variable: dltt
Defaulted Records: 65.1045
Non-Defaulted Records: 25.5793

Variable: xint
Defaulted Records: 8.2094
Non-Defaulted Records: 3.8612

Variable: total_debt
Defaulted Records: 73.5602
Non-Defaulted Records: 32.9835

Variable: debt_to_capital
Defaulted Records: 0.4393
Non-Defaulted Records: 0.2280


In [ ]:
import numpy as np

print(np.__version__)

2.2.4


In [ ]:
!pip install --upgrade numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 78.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.0
    Uninstalling numpy-1.26.0:
      Successfully uninstalled numpy-1.26.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.4 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.4 which is incompatible.
pytensor 2.27.1 requires numpy<2,>=1.17.0, but you have numpy 2.2.4 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.4 which is incompatible.
langchain 0.3.19 requires numpy<2,>=1.26.4; python_version < "3.12", but you have numpy 2.2.4 which is incompatible.
thinc 8.2.5 requires numpy<2.0.0